In [1]:
import os
import gc
import numpy as np
import pandas as pd
import joblib
from datetime import datetime

from lightgbm import LGBMClassifier
import optuna
from prunedcv import PrunedCV

from codes.utils import import_data, cross_val_score_auc, reduce_mem_usage, fix_dtypes
from codes.fe_browser import latest
from codes.fe_emails import proton, mappings
from codes.fe_cards import stats
from codes.fe_date import dates
from codes.fe_relatives import divisions, divisions_float
from codes.fe_categorical import pairs, wtf, cat_limit, encode_cat
from codes.prepro import prepro
from codes.fe_users import users_stats

from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import make_pipeline
from sklearn.base import TransformerMixin

/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
DATA_PATH = '../input/'
SEARCH_PARAMS = True
N_FOLD = 5
BOOSTING = 'goss'
RANDOM_STATE = 43

In [3]:
X_train, X_test, sample_submission = import_data(DATA_PATH)

### Some Feature Engineering

drop columns, count encoding, aggregation, fillna

In [4]:
if os.path.isfile('features_train.pkl'):
    X_train = joblib.load('features_train.pkl')
    X_test = joblib.load('features_test.pkl')
    y_train = joblib.load('y_train.pkl')
    
else:

    print('fix_dtypes')
    X_train, X_test = fix_dtypes(X_train, X_test)
    print('users_stats')
    X_train, X_test = users_stats(X_train, X_test)
    print('latest')
    X_train, X_test = latest(X_train, X_test)
    print('proton')
    X_train, X_test = proton(X_train, X_test)
    print('nulls1')
    X_train['nulls1'] = X_train.isna().sum(axis=1)
    X_test['nulls1'] = X_test.isna().sum(axis=1)
    print('mappings')
    X_train, X_test = mappings(X_train, X_test)
    print('stats')
    X_train, X_test = stats(X_train, X_test)
    print('divisions')
    X_train, X_test = divisions(X_train, X_test)
    print('dates')
    X_train, X_test = dates(X_train, X_test)
    print('pairs')
    X_train, X_test = pairs(X_train, X_test)
    print('encode_cat')
    X_train, X_test = encode_cat(X_train, X_test)
    print('wtf')
    # X_train, X_test = wtf(X_train, X_test)
    print('y')
    y_train = X_train['isFraud'].copy()
    X_train = X_train.drop('isFraud', axis=1)
    print('divisions_float')
    X_train, X_test = divisions_float(X_train, X_test)
    print('prepro')
    X_train, X_test = prepro(X_train, X_test)
    print('reduce_mem_usage')
    # X_train = reduce_mem_usage(X_train)
    # X_test = reduce_mem_usage(X_test)
    print('np.inf')
    X_train[X_train == np.inf] = -1
    X_train[X_train == -np.inf] = -1
    X_train[X_train.isna()] = -1
    X_test[X_test == np.inf] = -1
    X_test[X_test == -np.inf] = -1
    X_test[X_test.isna()] = -1
    print('TransactionDT')
    X_test.drop(['TransactionDT'], axis=1, inplace=True)
    X_train.drop(['TransactionDT'], axis=1, inplace=True)
    
    joblib.dump(X_train, 'features_train.pkl')
    joblib.dump(X_test, 'features_test.pkl')
    joblib.dump(y_train, 'y_train.pkl')

fix_dtypes
users_stats


/Users/piotrgabrys/data/python3/ieee/codes/fe_users.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_data = pd.concat([train, test])


latest
proton
nulls1
mappings
stats
divisions
dates
pairs
encode_cat
wtf
y
divisions_float
774
prepro
reduce_mem_usage
np.inf
TransactionDT


In [ ]:
train_new_feats = joblib.load('train_feats.pkl')
test_new_feats = joblib.load('test_feats.pkl')

for col in train_new_feats.select_dtypes('category').columns:
    train_new_feats.loc[:, col] = train_new_feats.loc[:, col].astype('int')
    test_new_feats.loc[:, col] = test_new_feats.loc[:, col].astype('int')

print('np.inf')
train_new_feats[train_new_feats == np.inf] = -1
train_new_feats[train_new_feats == -np.inf] = -1
train_new_feats[train_new_feats.isna()] = -1
test_new_feats[test_new_feats == np.inf] = -1
test_new_feats[test_new_feats == -np.inf] = -1
test_new_feats[test_new_feats.isna()] = -1

print(train_new_feats.shape[1])
print(X_train.shape[1])
X_train = pd.concat([X_train, train_new_feats], axis=1)
print(X_train.shape[1])
X_test = pd.concat([X_test, test_new_feats], axis=1)
del train_new_feats, test_new_feats

### Model and training

In [8]:
sel_mod = LGBMClassifier(metric='auc', boosting_type=BOOSTING)
sfm = SelectFromModel(sel_mod, threshold=0.5)
print(X_train.shape[1])
sfm.fit(X_train, y_train)
columns = list(X_train.columns[sfm.get_support()])
print(len(columns))
X_train = X_train.loc[:,columns]
X_test = X_test.loc[:,columns]

1264
858


In [ ]:
# 854
# 481

In [9]:
class Counter(TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print(X.shape[1])
        return X

In [10]:
sel_mod = LGBMClassifier(metric='auc', n_estimators=200, boosting_type=BOOSTING)
sel_mod.set_params(**pars)
model = make_pipeline(
    SelectFromModel(sel_mod),
    Counter(),
    LGBMClassifier(metric='auc',
                   boosting_type=BOOSTING,
                   n_estimators=2000)
)

In [11]:
prun = PrunedCV(N_FOLD, 0.02, minimize=False)

In [12]:
def objective(trial):
    
    joblib.dump(study, 'study_{}.pkl'.format(BOOSTING)) 

    
    params = {
        'selectfrommodel__threshold': trial.suggest_int('selectfrommodel__threshold', 1, 200),
        'lgbmclassifier__num_leaves': trial.suggest_int('lgbmclassifier__num_leaves', 10, 1500), 
        'lgbmclassifier__subsample_for_bin': trial.suggest_int('lgbmclassifier__subsample_for_bin', 1000, 5000000), 
        'lgbmclassifier__min_child_samples': trial.suggest_int('lgbmclassifier__min_child_samples', 200, 100000), 
        'lgbmclassifier__reg_alpha': trial.suggest_loguniform('lgbmclassifier__reg_alpha', 0.00000000001, 10.0),
        'lgbmclassifier__colsample_bytree': trial.suggest_loguniform('lgbmclassifier__colsample_bytree', 0.0001, 1.0),
        'lgbmclassifier__learning_rate': trial.suggest_loguniform('lgbmclassifier__learning_rate', 0.00001, 2.0)
    }
    
    print(params)
    
    model.set_params(**params)
    return prun.cross_val_score(model, 
                                X_train, 
                                y_train, 
                                metric='auc', 
                                shuffle=True, 
                                random_state=RANDOM_STATE)

In [ ]:
if SEARCH_PARAMS:
    if os.path.isfile('study_{}.pkl'.format(BOOSTING)):
        study = joblib.load('study_{}.pkl'.format(BOOSTING))
    else:
        study = optuna.create_study()

    study.optimize(objective, timeout=60 * 60 * 22)
    joblib.dump(study, 'study_{}.pkl'.format(BOOSTING))
    best_params = study.best_params

else:

    best_params = {
        'selectfrommodel__threshold': 20,
        'lgbmclassifier__num_leaves': 330,
        'lgbmclassifier__subsample_for_bin': 2077193,
        'lgbmclassifier__min_child_samples': 2227,
        'lgbmclassifier__reg_alpha': 0.16758905622425835,
        'lgbmclassifier__colsample_bytree': 0.49030006727392056,
        'lgbmclassifier__learning_rate': 0.07916040470631734
    }

{'selectfrommodel__threshold': 170, 'lgbmclassifier__num_leaves': 339, 'lgbmclassifier__subsample_for_bin': 1186152, 'lgbmclassifier__min_child_samples': 72164, 'lgbmclassifier__reg_alpha': 1.2579198177394756e-09, 'lgbmclassifier__colsample_bytree': 0.002257234008559371, 'lgbmclassifier__learning_rate': 0.1985648810166687}
37
37
39
39
39
39
38
38
40
40
36
36
40
40
39
39


[I 2019-09-13 15:04:38,489] Finished trial#0 resulted in value: -0.8163123545125364. Current best value is -0.8163123545125364 with parameters: {'selectfrommodel__threshold': 170, 'lgbmclassifier__num_leaves': 339, 'lgbmclassifier__subsample_for_bin': 1186152, 'lgbmclassifier__min_child_samples': 72164, 'lgbmclassifier__reg_alpha': 1.2579198177394756e-09, 'lgbmclassifier__colsample_bytree': 0.002257234008559371, 'lgbmclassifier__learning_rate': 0.1985648810166687}.


{'selectfrommodel__threshold': 80, 'lgbmclassifier__num_leaves': 964, 'lgbmclassifier__subsample_for_bin': 3809907, 'lgbmclassifier__min_child_samples': 70895, 'lgbmclassifier__reg_alpha': 0.020722815534357546, 'lgbmclassifier__colsample_bytree': 0.021012409267783357, 'lgbmclassifier__learning_rate': 0.5444138809014398}
126
126
125
125
124
124
126
126
127
127
128
128
129
129
127
127


[I 2019-09-13 15:34:46,783] Finished trial#1 resulted in value: -0.8996888834888637. Current best value is -0.8996888834888637 with parameters: {'selectfrommodel__threshold': 80, 'lgbmclassifier__num_leaves': 964, 'lgbmclassifier__subsample_for_bin': 3809907, 'lgbmclassifier__min_child_samples': 70895, 'lgbmclassifier__reg_alpha': 0.020722815534357546, 'lgbmclassifier__colsample_bytree': 0.021012409267783357, 'lgbmclassifier__learning_rate': 0.5444138809014398}.


{'selectfrommodel__threshold': 165, 'lgbmclassifier__num_leaves': 508, 'lgbmclassifier__subsample_for_bin': 1391520, 'lgbmclassifier__min_child_samples': 20661, 'lgbmclassifier__reg_alpha': 3.8077553579075474, 'lgbmclassifier__colsample_bytree': 0.7443367963866814, 'lgbmclassifier__learning_rate': 0.00022456711080226036}
41
41
40
40


...

{'selectfrommodel__threshold': 27, 'lgbmclassifier__num_leaves': 1379, 'lgbmclassifier__subsample_for_bin': 4361526, 'lgbmclassifier__min_child_samples': 11244, 'lgbmclassifier__reg_alpha': 1.0807896139262164e-09, 'lgbmclassifier__colsample_bytree': 0.9140938764571404, 'lgbmclassifier__learning_rate': 0.18912666104291284}
280
280
290
290
285
285
284
284
291
291
285
285
284
284
284
284


[I 2019-09-14 03:45:07,056] Finished trial#34 resulted in value: -0.9617737934032883. Current best value is -0.9617737934032883 with parameters: {'selectfrommodel__threshold': 27, 'lgbmclassifier__num_leaves': 1379, 'lgbmclassifier__subsample_for_bin': 4361526, 'lgbmclassifier__min_child_samples': 11244, 'lgbmclassifier__reg_alpha': 1.0807896139262164e-09, 'lgbmclassifier__colsample_bytree': 0.9140938764571404, 'lgbmclassifier__learning_rate': 0.18912666104291284}.


{'selectfrommodel__threshold': 2, 'lgbmclassifier__num_leaves': 1383, 'lgbmclassifier__subsample_for_bin': 4469896, 'lgbmclassifier__min_child_samples': 1292, 'lgbmclassifier__reg_alpha': 2.001811876397163e-09, 'lgbmclassifier__colsample_bytree': 0.9873066190734683, 'lgbmclassifier__learning_rate': 0.1793404007647492}
800
800
810
810
805
805
808
808
799
799
813
813
796
796
812
812


[I 2019-09-14 07:38:54,386] Finished trial#35 resulted in value: -0.9661841702672479. Current best value is -0.9661841702672479 with parameters: {'selectfrommodel__threshold': 2, 'lgbmclassifier__num_leaves': 1383, 'lgbmclassifier__subsample_for_bin': 4469896, 'lgbmclassifier__min_child_samples': 1292, 'lgbmclassifier__reg_alpha': 2.001811876397163e-09, 'lgbmclassifier__colsample_bytree': 0.9873066190734683, 'lgbmclassifier__learning_rate': 0.1793404007647492}.


{'selectfrommodel__threshold': 2, 'lgbmclassifier__num_leaves': 1406, 'lgbmclassifier__subsample_for_bin': 4596746, 'lgbmclassifier__min_child_samples': 2787, 'lgbmclassifier__reg_alpha': 1.9909124818304063e-09, 'lgbmclassifier__colsample_bytree': 0.4901999314700306, 'lgbmclassifier__learning_rate': 0.26666008814905623}
800
800
810
810
805
805
808
808
799
799
813
813
796
796
812


In [ ]:
model.set_params(**best_params)

cross_val_score_auc(model,
                    X_train,
                    y_train,
                    n_fold=N_FOLD,
                    stratify=True,
                    shuffle=True,
                    random_state=RANDOM_STATE,
                    predict=True,
                    X_test=X_test,
                    submission=sample_submission)

In [ ]:
# ROC accuracy: 0.9668942182909179, Train: 0.9999901167411397
# ROC accuracy: 0.9720552290202384, Train: 0.9999891233350843
# ROC accuracy: 0.9710663975253696, Train: 0.9999918268060299
# ROC accuracy: 0.9703005116766165, Train: 0.9999910116495871
# ROC accuracy: 0.9677524410936837, Train: 0.9999883123936292
# ROC accuracy: 0.970521434805755, Train: 0.9999753389326952
# ROC accuracy: 0.9709850608667766, Train: 0.9999787304381259
# ROC accuracy: 0.9708245135815027, Train: 0.9999796449943333


# 0.9700499758576075